### Demo: return longest string from list

In [0]:
def find_longest_string(list_of_strings):
  longest_string = None
  longest_string_len = 0
  for s in list_of_strings:
    if len(s) > longest_string_len:
      longest_string_len = len(s)
      longest_string = s
  return longest_string

In [0]:
list_of_strings = ['abc', 'python', 'dima']
%time max_length = print(find_longest_string(list_of_strings))

python
CPU times: user 1.03 ms, sys: 142 µs, total: 1.18 ms
Wall time: 1.09 ms


In [0]:
large_list_of_strings = list_of_strings * 1000
%time print(find_longest_string(large_list_of_strings))

python
CPU times: user 1.57 ms, sys: 0 ns, total: 1.57 ms
Wall time: 1.5 ms


In [0]:
large_list_of_strings = list_of_strings * 100000000
%time max_length = max(large_list_of_strings, key=len)

CPU times: user 17.2 s, sys: 111 ms, total: 17.3 s
Wall time: 25.1 s


### Break up into steps

- Step 1: Compute the length of all strings in a list
- Step 2: Select the maximum value

Note: Calculating lengths in one line and then zipping together with `list_of_strings` is faster than the alternative of appending to a list while physically passing through it.

In [0]:
%%time

# step 1: compute len of all strings in list
list_of_string_lens = [len(s) for s in list_of_strings]
list_of_string_lens = zip(list_of_strings, list_of_string_lens)

# step 2: select the maximum value
max_len = max(list_of_string_lens, key=lambda t: t[1])
print(max_len)

('python', 6)
CPU times: user 0 ns, sys: 1.15 ms, total: 1.15 ms
Wall time: 1.45 ms


This is slower than before, as we're passing through list twice - first time to get the length and the second time to get the maximum. **But** now the input to step 2 is some pre-processed data (not the original input)..

Introduce helper functions:
- `mapper`: given a string, returns its length
- `reducer`: gets two tuples and returns the one with the biggest length

In [0]:
# mapper gets a string and returns its length
mapper = len

# reducer gets two tuples and returns the one with the bigger length
def reducer(p, c):
  if p[1] > c[1]:
    return p
  return c

Rewrite code using `mapper` and `reducer`, making use of Python's functions (`map` and `reduce`).

In [0]:
# We'll use Python's reduce function, which needs importing in Python 3
from functools import reduce

In [0]:
%%time

#step 1
mapped = map(mapper, list_of_strings)
mapped = zip(list_of_strings, mapped)

#step 2:
reduced = reduce(reducer, mapped)
print(reduced)

('python', 6)
CPU times: user 619 µs, sys: 56 µs, total: 675 µs
Wall time: 2.81 ms


- Step 1: maps the list of strings into a list of tuples using the `mapper` function (use `zip` again to avoid duplicating the strings).
- Step 2: uses the `reducer` function, goes over the tuples from step one and applies it one by one. The result is a tuple with the maximum length.

### Break input into chunks

Run on the input divided up into chunks.

In [0]:
from math import ceil

# Write function that takes a list l and produces n chunks from it

def chunks(l, n):
  """Yield n successive chunks from l."""
  list_len = len(l)
  chunk_length = ceil(list_len / n)
  for i in range(0, list_len, chunk_length):
    yield l[i:i + chunk_length]

In [0]:
num_wanted_chunks = 32
# data_chunks is a generator, i.e. it's single use
data_chunks = chunks(large_list_of_strings, num_wanted_chunks)

In [0]:
%%time
#step 1:
reduced_all = []
for chunk in data_chunks:
  mapped_chunk = map(mapper, chunk)
  mapped_chunk = zip(chunk, mapped_chunk)
    
  reduced_chunk = reduce(reducer, mapped_chunk)
  reduced_all.append(reduced_chunk)
    
#step 2:
reduced = reduce(reducer, reduced_all)
print(reduced)

('python', 6)
CPU times: user 55.8 s, sys: 1.41 s, total: 57.2 s
Wall time: 58.6 s


Improvement: Convert `reduce` to work on parallel data too

In [0]:
def chunks_mapper(chunk):
  mapped_chunk = map(mapper, chunk) 
  mapped_chunk = zip(chunk, mapped_chunk)
  return reduce(reducer, mapped_chunk)

In [0]:
num_wanted_chunks = 32
# data_chunks is a generator, i.e. it's single use
data_chunks = chunks(large_list_of_strings, num_wanted_chunks)

In [0]:
%%time

#step 1:
mapped = map(chunks_mapper, data_chunks)

#step 2:
reduced = reduce(reducer, mapped)
print(reduced)

('python', 6)
CPU times: user 52.8 s, sys: 1.36 s, total: 54.2 s
Wall time: 55 s


Same sort of execution time, **but** we can now parallelise this!

### Parallelize using multiprocessing

We'll use Python's `multiprocessing` library, which splits execution over the processors you have available on your machine. There's a convenient `pool.map` function you can use instead of the regular `map` function.

In [0]:
# Should choose chunk size based on your number of processors
data_chunks = chunks(large_list_of_strings, 32)

In [0]:
%%time
from multiprocessing import Pool

pool = Pool(8)

#step 1:
mapped = pool.map(chunks_mapper, data_chunks)

#step 2:
reduced = reduce(reducer, mapped)
print(reduced)

('python', 6)
CPU times: user 14.9 s, sys: 1.81 s, total: 16.7 s
Wall time: 1min 13s
